**Cat and Dog classiffier using pytorch**

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Assuming you have already downloaded and extracted the dataset
train_dir = '/content/drive/MyDrive/archive (1)/training_set/training_set'
test_dir = '/content/drive/MyDrive/archive (1)/test_set/test_set'

# Define transforms for training and validation datasets
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset using ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class CatDogClassifier(nn.Module):
    def __init__(self):
        super(CatDogClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 30, kernel_size=3, padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(30, 30, kernel_size=3, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(30, 30, kernel_size=3, padding=0)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(30 * 30 * 30, 128)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(64, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = x.view(-1, 30 * 30 * 30)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Create an instance of the model and transfer it to the device
model = CatDogClassifier().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    train_losses = []
    train_accs = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')

    return model, train_losses, train_accs

# Function to validate the model
def validate_model(model, test_loader, criterion, device='cpu'):
    model.eval()
    val_losses = []
    val_accs = []
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            val_losses.append(loss.item())

        val_acc = correct / total
        val_accs.append(val_acc)

        print(f'Validation Loss: {np.mean(val_losses):.4f}, Validation Acc: {val_acc:.4f}')

    return val_acc, val_losses

# Train the model
model, train_losses, train_accs = train_model(model, train_loader, criterion, optimizer, num_epochs=10, device=device)

# Validate the model
val_acc, val_losses = validate_model(model, test_loader, criterion, device=device)

# Save the model
save_path = 'cat_dog_classifier.pth'  # Specify the path where you want to save the model
torch.save(model.state_dict(), save_path)

# Function to predict on a single image
def predict_image(model, image_path, device='cpu'):
    model.eval()
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))
    image = transforms.ToTensor()(image)
    image = image.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    if predicted.item() == 0:
        print("Prediction: Cat")
    else:
        print("Prediction: Dog")

    # Display the resized image using OpenCV and Matplotlib
    plt.imshow(cv2.cvtColor(cv2.resize(cv2.imread(image_path), (256, 256)), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Example usage of predict_image function
test_image_path = '/content/dog.jpeg'  # Replace with the path to your test image
predict_image(model, test_image_path, device=device)

# Plot training and validation accuracy values
plt.plot(train_accs, color='red', label='Train')
plt.plot([0, len(train_accs)-1], [val_acc, val_acc], linestyle='--', color='blue', label='Validation')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss values
plt.plot(train_losses, color='red', label='Train')
plt.plot(val_losses, color='blue', label='Validation')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


Epoch [1/10], Train Loss: 0.6880, Train Acc: 0.5525
Epoch [2/10], Train Loss: 0.6440, Train Acc: 0.6341
Epoch [3/10], Train Loss: 0.5976, Train Acc: 0.6952
Epoch [4/10], Train Loss: 0.5456, Train Acc: 0.7556
Epoch [5/10], Train Loss: 0.4970, Train Acc: 0.8076
Epoch [6/10], Train Loss: 0.4464, Train Acc: 0.8655
Epoch [7/10], Train Loss: 0.4136, Train Acc: 0.8984
